In [5]:
!pip install --upgrade pip
!pip install pmdarima
!pip install psycopg2-binary
!pip install mpld3

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at 

In [6]:
#Import Python Packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
from datetime import datetime, timedelta
import time
import psycopg2
import boto3

from statsmodels.tsa.seasonal import seasonal_decompose 
# holt winters 
# single exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
# double and triple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False
import mpld3
mpld3.enable_notebook()

In [7]:
# Put the ARN of your AWS Secrets Manager secret for your redshift cluster here:
secret_name="arn:aws:secretsmanager:us-west-2:451614664738:secret:ForecastingSecret-JRCdLL"

# This will get the secret from AWS Secrets Manager. 
import boto3
import json

session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager'
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

if 'SecretString' in get_secret_value_response:
    connection_info = json.loads(get_secret_value_response['SecretString'])
else:
    print("ERROR: no secret data found")

In [8]:
# assign a variable that contains a string of your credentials
credentials=psycopg2.connect(
    dbname   = 'exelredshift', 
    host     = connection_info["host"], 
    port     = connection_info["port"], 
    user     = connection_info["username"], 
    password = connection_info["password"]
)

In [10]:
# read in your SQL query results using pandas
df = pd.read_sql("""
            select * from edm.bl_cabo_rcc_hcc_nps
            """, con = credentials)


In [7]:
selection_rcc_comm = df[['week_ending_dt','projected_sp_rcc_sales','projected_sd_rcc_sales']]
selection_hcc_comm = df[['week_ending_dt','projected_sp_hcc_sales','projected_sd_hcc_sales']]
selection_dtc_comm = df[['week_ending_dt','projected_sp_thy_sales','projected_sd_thy_sales']]
selection_ease = df[['week_ending_dt','projected_ease_rcc_sales','projected_ease_hcc_sales','projected_ease_thy_sales']]

In [8]:

sorted_data_rcc_comm= selection_rcc_comm.sort_values(by='week_ending_dt')
sorted_data_hcc_comm= selection_hcc_comm.sort_values(by='week_ending_dt')
sorted_data_dtc_comm= selection_dtc_comm.sort_values(by='week_ending_dt')
sorted_data_ease = selection_ease.sort_values(by='week_ending_dt')


In [9]:
rcc_agg_df= sorted_data_rcc_comm.groupby(["week_ending_dt"]).sum()
hcc_agg_df= sorted_data_hcc_comm.groupby(["week_ending_dt"]).sum()
dtc_agg_df= sorted_data_dtc_comm.groupby(["week_ending_dt"]).sum()
ease_agg_df= sorted_data_ease.groupby(["week_ending_dt"]).sum()


In [10]:
rcc_agg_sorted = rcc_agg_df.sort_values(by='week_ending_dt')
hcc_agg_sorted = hcc_agg_df.sort_values(by='week_ending_dt')
dtc_agg_sorted = dtc_agg_df.sort_values(by='week_ending_dt')
ease_agg_sorted = ease_agg_df.sort_values(by='week_ending_dt')

In [11]:
rcc_dfn= rcc_agg_sorted.reset_index()
hcc_dfn= hcc_agg_sorted.reset_index()
dtc_dfn= dtc_agg_sorted.reset_index()
ease_dfn= ease_agg_sorted.reset_index()

In [12]:

rcc_dfn['RCC_Outs'] = rcc_dfn['projected_sp_rcc_sales']+ rcc_dfn['projected_sd_rcc_sales']
hcc_dfn['HCC_Outs'] = hcc_dfn['projected_sp_hcc_sales']+ hcc_dfn['projected_sd_hcc_sales']
dtc_dfn['DTC_Outs'] = dtc_dfn['projected_sp_thy_sales']+ dtc_dfn['projected_sd_thy_sales']
ease_dfn['EASE_Outs'] = ease_dfn['projected_ease_rcc_sales']+ ease_dfn['projected_ease_hcc_sales']+ ease_dfn['projected_ease_thy_sales']



In [13]:

rcc_new_dfn= rcc_dfn[['week_ending_dt', 'RCC_Outs']]
hcc_new_dfn= hcc_dfn[['week_ending_dt', 'HCC_Outs']]
dtc_new_dfn= dtc_dfn[['week_ending_dt', 'DTC_Outs']]
ease_new_dfn= ease_dfn[['week_ending_dt', 'EASE_Outs']]


In [14]:
#Convert to datetime format
rcc_new_dfn['week_end_date'] = pd.to_datetime(rcc_new_dfn['week_ending_dt'])
hcc_new_dfn['week_end_date'] = pd.to_datetime(hcc_new_dfn['week_ending_dt'])
dtc_new_dfn['week_end_date'] = pd.to_datetime(dtc_new_dfn['week_ending_dt'])
ease_new_dfn['week_end_date'] = pd.to_datetime(ease_new_dfn['week_ending_dt'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [15]:
rcc_sorted_dfn= rcc_new_dfn.sort_values(by='week_end_date')
rcc_sorted_dfn.reset_index()

hcc_sorted_dfn= hcc_new_dfn.sort_values(by='week_end_date')
hcc_sorted_dfn.reset_index()

dtc_sorted_dfn= dtc_new_dfn.sort_values(by='week_end_date')
dtc_sorted_dfn.reset_index()

ease_sorted_dfn= ease_new_dfn.sort_values(by='week_end_date')
ease_sorted_dfn.reset_index()

,index,week_ending_dt,EASE_Outs,week_end_date
0,0,2018-01-05,15.50,2018-01-05
1,1,2018-01-12,41.50,2018-01-12
2,2,2018-01-19,37.97,2018-01-19
3,3,2018-01-26,38.96,2018-01-26
4,4,2018-02-02,69.50,2018-02-02
...,...,...,...,...
285,285,2023-06-23,155.50,2023-06-23
286,286,2023-06-30,195.50,2023-06-30
287,287,2023-07-07,149.00,2023-07-07
288,288,2023-07-14,160.50,2023-07-14


In [16]:
rcc_master_dataframe_comm= rcc_sorted_dfn[['week_end_date','RCC_Outs']]
hcc_master_dataframe_comm= hcc_sorted_dfn[['week_end_date','HCC_Outs']]
dtc_master_dataframe_comm= dtc_sorted_dfn[['week_end_date','DTC_Outs']]
ease_master_dataframe_comm= ease_sorted_dfn[['week_end_date','EASE_Outs']]

In [17]:
rcc_master_dataframe_comm['RCC_Outs'] = rcc_master_dataframe_comm['RCC_Outs'].apply(np.ceil)
hcc_master_dataframe_comm['HCC_Outs'] = hcc_master_dataframe_comm['HCC_Outs'].apply(np.ceil)
dtc_master_dataframe_comm['DTC_Outs'] = dtc_master_dataframe_comm['DTC_Outs'].apply(np.ceil)
ease_master_dataframe_comm['EASE_Outs'] = ease_master_dataframe_comm['EASE_Outs'].apply(np.ceil)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [18]:
rcc_master_dataframe_comm = rcc_master_dataframe_comm.astype({"RCC_Outs": int})
hcc_master_dataframe_comm = hcc_master_dataframe_comm.astype({"HCC_Outs": int})
dtc_master_dataframe_comm = dtc_master_dataframe_comm.astype({"DTC_Outs": int})
ease_master_dataframe_comm = ease_master_dataframe_comm.astype({"EASE_Outs": int})

In [19]:
rcc_master_dataframe_cabo = rcc_master_dataframe_comm[['week_end_date','RCC_Outs']]
hcc_master_dataframe_cabo = hcc_master_dataframe_comm[['week_end_date','HCC_Outs']]
dtc_master_dataframe_cabo = dtc_master_dataframe_comm[['week_end_date','DTC_Outs']]
ease_master_dataframe_cabo = ease_master_dataframe_comm[['week_end_date','EASE_Outs']]

In [20]:
rcc_master_dataframe_cabo['week_end_date']=pd.to_datetime(rcc_master_dataframe_cabo['week_end_date'])
hcc_master_dataframe_cabo['week_end_date']=pd.to_datetime(hcc_master_dataframe_cabo['week_end_date'])
dtc_master_dataframe_cabo['week_end_date']=pd.to_datetime(dtc_master_dataframe_cabo['week_end_date'])
ease_master_dataframe_cabo['week_end_date']=pd.to_datetime(ease_master_dataframe_cabo['week_end_date'])

In [21]:
rcc_new_df1 = rcc_master_dataframe_cabo[rcc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
hcc_new_df1 = hcc_master_dataframe_cabo[hcc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
dtc_new_df1 = dtc_master_dataframe_cabo[dtc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
ease_new_df1 = ease_master_dataframe_cabo[ease_master_dataframe_cabo['week_end_date'] > '2021-01-01']

In [22]:
#Set date column as the index
rcc_new_df =rcc_new_df1.set_index('week_end_date')
hcc_new_df =hcc_new_df1.set_index('week_end_date')
dtc_new_df =dtc_new_df1.set_index('week_end_date')
ease_new_df =ease_new_df1.set_index('week_end_date')

In [23]:
rcc_a = rcc_master_dataframe_cabo.tail(1)
hcc_a = hcc_master_dataframe_cabo.tail(1)
dtc_a = dtc_master_dataframe_cabo.tail(1)
ease_a = ease_master_dataframe_cabo.tail(1)

In [24]:

rcc_train  = rcc_new_df[(rcc_new_df.index.get_level_values(0) >= rcc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (rcc_new_df.index.get_level_values(0) < rcc_a['week_end_date'].iloc[0])]

hcc_train  = hcc_new_df[(hcc_new_df.index.get_level_values(0) >= hcc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (hcc_new_df.index.get_level_values(0) < hcc_a['week_end_date'].iloc[0])]

dtc_train  = dtc_new_df[(dtc_new_df.index.get_level_values(0) >= dtc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (dtc_new_df.index.get_level_values(0) < dtc_a['week_end_date'].iloc[0])]

ease_train = ease_new_df[(ease_new_df.index.get_level_values(0) >= ease_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (ease_new_df.index.get_level_values(0) < ease_a['week_end_date'].iloc[0])]


In [25]:
#RCC Model

rcc_fitted_model = ExponentialSmoothing(rcc_train['RCC_Outs'],trend='mul',seasonal='mul',seasonal_periods=12).fit()
rcc_test_predictions = rcc_fitted_model.forecast(52)
rcc_train['RCC_Outs'].plot(legend=True,label='TRAIN')
rcc_test_predictions.plot(legend=True,label='PREDICTION')
plt.title('Train, Test and Predicted Test using Holt Winters')



/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


Text(0.5, 1.0, 'Train, Test and Predicted Test using Holt Winters')

In [26]:
#HCC Model

hcc_fitted_model = ExponentialSmoothing(hcc_train['HCC_Outs'],trend='mul',seasonal='mul',seasonal_periods=12).fit()
hcc_test_predictions = hcc_fitted_model.forecast(52)
hcc_train['HCC_Outs'].plot(legend=True,label='TRAIN')
hcc_test_predictions.plot(legend=True,label='PREDICTION')
plt.title('Train, Test and Predicted Test using Holt Winters')


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)


Text(0.5, 1, 'Train, Test and Predicted Test using Holt Winters')

In [27]:
#DTC Model

dtc_fitted_model = ExponentialSmoothing(dtc_train['DTC_Outs'],trend='add',seasonal='add',seasonal_periods=12).fit()
dtc_test_predictions = dtc_fitted_model.forecast(52)
dtc_train['DTC_Outs'].plot(legend=True,label='TRAIN')
dtc_test_predictions.plot(legend=True,label='PREDICTION')
plt.title('Train, Test and Predicted Test using Holt Winters')



/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)


Text(0.5, 1, 'Train, Test and Predicted Test using Holt Winters')

In [28]:
#EASE Model

ease_fitted_model = ExponentialSmoothing(ease_train['EASE_Outs'],trend='mul',seasonal='mul',seasonal_periods=12).fit()
ease_test_predictions = ease_fitted_model.forecast(52)
ease_train['EASE_Outs'].plot(legend=True,label='TRAIN')
ease_test_predictions.plot(legend=True,label='PREDICTION')
plt.title('Train, Test and Predicted Test using Holt Winters')


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


Text(0.5, 1, 'Train, Test and Predicted Test using Holt Winters')

In [29]:
from datetime import date
a = date(date.today().year, 1, 1)

In [30]:
forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')

In [31]:
#RCC
rcc_forecast1_df = pd.DataFrame(rcc_test_predictions,index= forecast_range, columns=['Prediction'])
rcc_forecast1_df.dropna(inplace=True)

rcc_future_forecasts= pd.concat([rcc_new_df['RCC_Outs'].iloc[:-1],rcc_forecast1_df],axis=1)

In [32]:
#HCC
hcc_forecast1_df = pd.DataFrame(hcc_test_predictions,index= forecast_range, columns=['Prediction'])
hcc_forecast1_df.dropna(inplace=True)

hcc_future_forecasts= pd.concat([hcc_new_df['HCC_Outs'].iloc[:-1],hcc_forecast1_df],axis=1)

In [33]:
#DTC 
dtc_forecast1_df = pd.DataFrame(dtc_test_predictions,index= forecast_range, columns=['Prediction'])
dtc_forecast1_df.dropna(inplace=True)

dtc_future_forecasts= pd.concat([dtc_new_df['DTC_Outs'].iloc[:-1],dtc_forecast1_df],axis=1)

In [34]:
ease_forecast1_df = pd.DataFrame(ease_test_predictions,index= forecast_range, columns=['Prediction'])
ease_forecast1_df.dropna(inplace=True)

ease_future_forecasts= pd.concat([ease_new_df['EASE_Outs'].iloc[:-1],ease_forecast1_df],axis=1)

In [35]:

rcc_future_forecasts['Forecast_totals'] = rcc_future_forecasts.fillna(0)['RCC_Outs'] + rcc_future_forecasts.fillna(0)['Prediction']
hcc_future_forecasts['Forecast_totals'] = hcc_future_forecasts.fillna(0)['HCC_Outs'] + hcc_future_forecasts.fillna(0)['Prediction']
dtc_future_forecasts['Forecast_totals'] = dtc_future_forecasts.fillna(0)['DTC_Outs'] + dtc_future_forecasts.fillna(0)['Prediction']
ease_future_forecasts['Forecast_totals'] = ease_future_forecasts.fillna(0)['EASE_Outs'] + ease_future_forecasts.fillna(0)['Prediction']


In [36]:
s_e=rcc_train
s_e=s_e.reset_index()
s= s_e.head(1)
e= s_e.tail(1)

In [37]:
#RCC

rcc_df_Fut = rcc_future_forecasts.reset_index()
rcc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
rcc_df_Fut['Year'] = pd.PeriodIndex(rcc_df_Fut.Date, freq='Y')
rcc_df_Fut['Quarter'] = rcc_df_Fut.Date.dt.quarter
weekend= rcc_master_dataframe_cabo.tail(2)
rcc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
rcc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
rcc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
rcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
rcc_df_Fut['Model_Type'] = 'RCC-HOLT'



In [38]:
#HCC

hcc_df_Fut = hcc_future_forecasts.reset_index()
hcc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
hcc_df_Fut['Year'] = pd.PeriodIndex(hcc_df_Fut.Date, freq='Y')
hcc_df_Fut['Quarter'] = hcc_df_Fut.Date.dt.quarter
weekend= hcc_master_dataframe_cabo.tail(2)
hcc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
hcc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
hcc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
hcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
hcc_df_Fut['Model_Type'] = 'HCC-HOLT'



In [39]:
#DTC

dtc_df_Fut = dtc_future_forecasts.reset_index()
dtc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
dtc_df_Fut['Year'] = pd.PeriodIndex(dtc_df_Fut.Date, freq='Y')
dtc_df_Fut['Quarter'] = dtc_df_Fut.Date.dt.quarter
weekend= dtc_master_dataframe_cabo.tail(2)
dtc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
dtc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
dtc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
dtc_df_Fut['Sales_Type'] = 'COMMERCIAL'
dtc_df_Fut['Model_Type'] = 'DTC-HOLT'



In [40]:
#EASE

ease_df_Fut = ease_future_forecasts.reset_index()
ease_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
ease_df_Fut['Year'] = pd.PeriodIndex(ease_df_Fut.Date, freq='Y')
ease_df_Fut['Quarter'] = ease_df_Fut.Date.dt.quarter
weekend= ease_master_dataframe_cabo.tail(2)
ease_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
ease_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
ease_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
ease_df_Fut['Sales_Type'] = 'EASE'
ease_df_Fut['Model_Type'] = 'EASE-HOLT'



In [41]:

rcc_hcc_one = rcc_df_Fut.append(hcc_df_Fut)
rcc_hcc_dtc = rcc_hcc_one.append(dtc_df_Fut)
all_one = rcc_hcc_dtc.append(ease_df_Fut)


In [42]:
final_df= all_one[['Date','Year','Quarter','Week_Ending_date','Sales_Type','Model_Type','Training_Start_date','Training_End_date','RCC_Outs','HCC_Outs','DTC_Outs','EASE_Outs','Prediction','Forecast_totals']]

In [43]:
final_df.reset_index(drop=True, inplace=True)

In [44]:
s_e=final_df
s_e=s_e.reset_index()
e= s_e.Week_Ending_date.iloc[0]

In [45]:
file_name_1 = "HOLT_WINTERS_FORECAST.csv"
final_df.to_csv(file_name_1,index=False)

In [46]:
a = 'HOLT_WINTERS_FORECAST.csv'
b = str(e)

In [47]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name_1, 'forecastingpoc', 'inputfiles/'+b+' - '+a)